In [1]:
import sys
# !{sys.executable} -m pip install h5py
# Suppress PySCF warning...
import pyscf
pyscf.__config__.B3LYP_WITH_VWN5 = False
from pathlib import Path
from pyscf.gto import Mole
from pyscf.lib import logger
# Wrapper functions to perform selection for variable and fixed active space sizes
from asf.wrapper import find_from_mol, find_from_scf, sized_space_from_mol, sized_space_from_scf

# Various utility functions...
from asf.utility import compare_active_spaces, show_mos_grid, pictures_Jmol
from pyscf import gto, mp, mcscf, mrpt, dmrgscf,scf

# Functionality for (state-averaged) CASSCF.
from pyscf.mcscf import CASSCF, state_average_mix
from pyscf.fci.direct_spin1 import FCISolver
from pyscf.fci.addons import fix_spin


In [2]:
# from pyscf.gto import Mole
# from pyscf.lib import logger
# from asf.wrapper import find_from_mol, sized_space_from_mol

ethene_geometry = """
    C        0.6695      0.0000      0.0000
    C       -0.6695      0.0000      0.0000
    H        1.2320      0.0000     -0.9289
    H        1.2320      0.0000      0.9289
    H       -1.2320      0.0000      0.9289
    H       -1.2320      0.0000     -0.9289
"""
# iron_geometry = """
# O      2.09945   -0.20712   -0.06955
# H      2.65808   -0.11922   -0.85974
# H      2.66428   -0.52573    0.65202
# O      0.04751    0.32344   -1.70791
# O     -2.09702    0.08054   -0.09941
# N      0.13864    2.10970    0.23028
# O     -0.16040   -2.04758   -0.25208
# O     -0.05207   -0.35287    1.90361
# H     -2.69555   -0.01745    0.65788
# H     -2.63984    0.28975   -0.87742
# H     -0.14720    2.62871   -0.60801
# H      1.10480    2.39390    0.42897
# H     -0.98651   -2.36883   -0.65341
# H      0.54518   -2.50373   -0.74261
# H     -0.11214   -1.27936    2.18843
# H      0.12164    0.17676    2.69663
# Fe     0.01013    0.06693   -0.14210
# H     -0.44640    2.44594    1.00267
# """


mol = Mole(atom=ethene_geometry, basis="def2-SVP", charge=0, spin=0, verbose=logger.INFO).build()
# mol = Mole(atom=iron_geometry, basis="def2-SVP", charge=2, spin=2, verbose=logger.INFO).build()

# Find one active space.
active_space = find_from_mol(mol)

System: uname_result(system='Linux', node='debian', release='6.10.4-amd64', version='#1 SMP PREEMPT_DYNAMIC Debian 6.10.4-1 (2024-08-12)', machine='x86_64')  Threads 16
Python 3.10.4 (main, Mar 31 2022, 08:41:55) [GCC 7.5.0]
numpy 1.22.3  scipy 1.7.3  h5py 3.12.1
Date: Thu Oct  3 18:35:19 2024
PySCF version 2.6.2
PySCF path  /home/grierjones/miniconda3/envs/DDGNN/lib/python3.10/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 16
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 C      0.669500000000   0.000000000000   0.000000000000 AA    1.265171640396   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 C     -0.669500000000   0.000000000000   0.000000000000 AA   -1.265171640396   0.000000000000   

ValueError: Insufficient precision in available types to represent (63, 52, 11, 0, 52)

In [ ]:
from asf.asfbase import print_mo_table
from asf.filters import ActiveSpaceSelectionError
from asf.scf import stable_scf
from asf.preselection import MP2PairinfoPreselection, MP2NatorbPreselection
from asf.utility import pictures_Jmol, show_mos_grid
from asf import ASFDMRG, ASFCI
from pathlib import Path
from pprint import pprint
from pyscf.gto import M

mf = stable_scf(mol)
sf = ASFDMRG.from_preselection(MP2NatorbPreselection(mf), maxM=150)
print(f"-> Selected an initial space of {sf.nel} electrons and {sf.norb} orbitals.")
sf.calculate()

In [ ]:
active_space.to_dict()

In [ ]:
sized_space_from_mol(mol, size=4)
